# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Car_Evaluation_Hyperdrive'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cpu_cluster = ComputeTarget(workspace=ws, name="capstone-compute")
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,"project-cluster", compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [4]:
%%writefile conda_dependencies.yml
 dependencies:
 - python=3.6.2
 - scikit-learn
 - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
env = Environment.from_conda_specification(name="sklearn-env", file_path="./conda_dependencies.yml")

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--n_estimators': choice(50,100,150,200,250),
        '--max_depth': choice(10,20,30,40,50),
        '--criterion': choice("gini","entropy"),
        '--max_features': choice(5,10,15)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = ".", compute_target = cpu_cluster, script = 'train.py', environment = env)

hyperdrive_run_config = HyperDriveConfig(run_config = estimator, 
                             hyperparameter_sampling = param_sampling,
                             policy = early_termination_policy,
                             primary_metric_name = 'Accuracy', 
                             primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs = 20,
                             max_concurrent_runs = 5)

In [6]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: 6f27140d-1ca1-479e-a502-dd6c7e627e52
Web View: https://ml.azure.com/experiments/Car_Evaluation_Hyperdrive/runs/6f27140d-1ca1-479e-a502-dd6c7e627e52?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-136554/workspaces/quick-starts-ws-136554

Execution Summary
RunId: 6f27140d-1ca1-479e-a502-dd6c7e627e52
Web View: https://ml.azure.com/experiments/Car_Evaluation_Hyperdrive/runs/6f27140d-1ca1-479e-a502-dd6c7e627e52?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-136554/workspaces/quick-starts-ws-136554



{'runId': '6f27140d-1ca1-479e-a502-dd6c7e627e52',
 'target': 'local',
 'status': 'Canceled',
 'startTimeUtc': '2021-01-30T07:44:34.670639Z',
 'endTimeUtc': '2021-01-30T07:55:25.159Z',
 'properties': {'ContentSnapshotId': '5422888d-0eb9-48e7-a4d3-902263287643'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {},
 'submittedBy': 'ODL_User 136554'}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print("Accuracy: {}".format(best_run_metrics['Accuracy']))

Accuracy: 0.9884393063583815


In [9]:
#TODO: Save the best model
best_run.register_model("car-evaluation-hyperdrive-best-model","outputs/capstone-hyperdrive-model.joblib")

Model(workspace=Workspace.create(name='quick-starts-ws-136554', subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30', resource_group='aml-quickstarts-136554'), name=car-evaluation-hyperdrive-best-model, id=car-evaluation-hyperdrive-best-model:3, version=3, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service